In [1]:
import sys
sys.path.insert(0, '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version')
sys.path.insert(0, '/home/steve/Dokumenty/Studia/project/CSGNet')

from ucsgnet.dataset import CADDataset, get_simple_2d_transforms
import matplotlib.pyplot as plt
from ucsgnet.ucsgnet.cad.net_cad import Net
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import math

In [2]:
params_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/models/pretrained/models/cad_main/initial/params.json'
weights_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/models/pretrained/models/cad_main/initial/ckpts/model.ckpt'
synthetic_dir = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/data/synthetic'
cad_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/data/cad/cad.h5'

In [3]:
dataset = CADDataset(cad_path, '', get_simple_2d_transforms())
loader = DataLoader(
            dataset=dataset,
            batch_size=1, # 16
            shuffle=False,
            drop_last=False,
            num_workers=0,
        )
len(dataset), len(loader)

(3000, 3000)

In [4]:
dataset[0][0].shape
# dataset[1][2]

torch.Size([1, 64, 64])

In [5]:
# net = Net.load_from_checkpoint(weights_path)
# net.eval()
# net.freeze()
# net.net
net = Net(
    
)

Num of trainable params: 5709491
Num of not trainable params: 0


RuntimeError: Error(s) in loading state_dict for Net:
	Missing key(s) in state_dict: "net.evaluator_.parts.2.layers_.weight", "net.evaluator_.parts.2.layers_.bias". 
	size mismatch for net.csg_layers_.0.composition_vector_1_: copying a param with shape torch.Size([256, 32, 4]) from checkpoint, the shape in current model is torch.Size([256, 48, 4]).
	size mismatch for net.csg_layers_.0.composition_vector_2_: copying a param with shape torch.Size([256, 32, 4]) from checkpoint, the shape in current model is torch.Size([256, 48, 4]).
	size mismatch for net.csg_layers_.0.parameter_encoder.0.weight: copying a param with shape torch.Size([256, 256]) from checkpoint, the shape in current model is torch.Size([256, 384]).
	size mismatch for net.csg_layers_.1.composition_vector_1_: copying a param with shape torch.Size([256, 48, 1]) from checkpoint, the shape in current model is torch.Size([256, 64, 1]).
	size mismatch for net.csg_layers_.1.composition_vector_2_: copying a param with shape torch.

In [ ]:
prev = None
counter = {'t': 0, 'f': 0}
for batch in tqdm(loader):
    image, points, trues, _ = batch
#     if prev is not None:
#         if torch.equal(prev, points):
#             counter['t'] += 1
#         else:
#             counter['f'] += 1
#     prev = points
    hidden = net.net.encoder_(image)
    out = net.net.decoder_(hidden)
#     out = net(image, points)
    points = points.unsqueeze(
        dim=1
    )  # broadcasting for different of shapes

    base_shapes_circle = net.net.evaluator_.parts[0](
        out, points
    )  # -> batch, num_shapes, num_points
    base_shapes_square = net.net.evaluator_.parts[1](
        out, points
    )

#     base_shapes = base_shapes.permute(
#         (0, 2, 1)
#     )  # -> batch, num_points, num_shapes
    break
# counter

In [ ]:
base_shapes_circle

In [ ]:
base_shapes_square

In [ ]:
# points.size(), points.min() + 0.0078

In [ ]:
# q_points = points.abs()
# q_points.max(torch.zeros_like(q_points)).norm(dim=-1)